## Header Files

In [178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from datetime import datetime
import seaborn as sns
from itertools import combinations
import math
import random
from scipy.stats import skew, kurtosis
import warnings
from PowerFlowAnalysis import PowerFlow
import time
from statistics import mean


# Suppress DeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Feature Engineering

In [179]:
data_file_path = 'Sample_Data8.xlsx' #INPUT
dist_file_path = 'Distance2.xlsx' #READY
ratings_file_path = 'Ratings.xlsx' #READY
line_data_path = 'Linedata2.csv' #READY
distdf = pd.read_excel(dist_file_path)
ratings_df = pd.read_excel(ratings_file_path)
line_data_df = pd.read_csv(line_data_path)
df = pd.read_excel(data_file_path)
df

,Period (Hr),Prosumer_Id,Solar,Storage,Total (Units),Load Forecast (Units),Unit Price,Profit_pref,Rating_pref,Solar Panel Rating,DSM
0,1,2,0.25625,0.830,1.08625,0.7521,10,0.32,0.68,2,0.000000
1,1,3,0.26250,0.750,1.01250,0.2300,7,0.16,0.84,2,0.000000
2,1,4,0.00000,0.000,0.00000,0.8700,10,0.31,0.69,3,0.485751
3,1,5,0.36250,0.658,1.02050,0.3140,7,0.83,0.17,3,0.111042


In [180]:
df['Value'] = round(df['Total (Units)'] - df['Load Forecast (Units)'],3)
df['Category'] = np.where(df['Value'] < 0, 'B', np.where(df['Value'] > 0, 'S', 'B'))
df['Value'] = np.where(df['Value'] > 0, df['Value'], -1 * df['Value'])
df['Profit_pref']=round(df['Profit_pref'],1)
df['Rating_pref']=round(df['Rating_pref'],1)
df['DSM'] = round(df['DSM'],0)
df.to_csv('df.csv', index=False)

# Find max_id
max_id = df['Prosumer_Id'].max()


#df

## Creating Bus

In [181]:
def create_bus_data(period_data):
    
#                   0   1   2   3    4    5   6   7  8     9
#                 |Bus|Type|Vp|theta|PGi|QGi|PLi|QLi|Qmin|Qmax|
    
    bus_data = []
    
    bus_data.append({'Bus': 1, 'Type': 1, 'Vp': 1, 'theta': 0, 'PGi': 0, 'QGi': 0, 'PLi': 0, 'QLi': 0, 'Qmin': 0, 'Qmax': 0})

    for index, row in period_data.iterrows():
        if row['Category'] == 'B':
            bus_data.append({'Bus': row['Prosumer_Id'], 'Type': 3, 'Vp': 1, 'theta': 0.0, 
                             'PGi': 0, 'QGi': 0, 'PLi': row['Value']*1000*4, 'QLi': 0, 'Qmin': 0, 'Qmax': 0})
        elif row['Category'] == 'S':
            bus_data.append({'Bus': row['Prosumer_Id'], 'Type': 3, 'Vp': 1, 'theta': 0.0, 
                             'PGi': 0, 'QGi': 0, 'PLi': 0, 'QLi': 0, 'Qmin': 0, 'Qmax': 0})
            
    bus_df = pd.DataFrame(bus_data)
    
    return bus_df

In [182]:
line_data_df

,From,To,Rij,Xij,Bij,a
0,1,2,0.001233,0.004127,0,1
1,2,3,0.002467,0.006051,0,1
2,2,4,0.007469,0.012050,0,1
3,4,5,0.006984,0.006084,0,1
4,2,6,0.019837,0.017276,0,1
5,6,7,0.009057,0.007886,0,1
6,7,8,0.020552,0.011640,0,1
7,7,9,0.047953,0.027160,0,1
8,3,10,0.053434,0.030264,0,1
9,5,10,0.001426,0.004522,0,1


In [183]:
# Delete rows where From or To is greater than max_id
line_data_df = line_data_df[(line_data_df['From'] <= max_id) & (line_data_df['To'] <= max_id)]

linedata = line_data_df.to_numpy()
BMva = 1e5  # Base VA

## SDR

In [184]:
mini_price = 3.6

In [185]:
sdr_df = pd.DataFrame(columns=['Period', 'SDR'])

for period in df['Period (Hr)'].unique():
    data = df[df['Period (Hr)'] == period]

    TBP = data[data['Category'] == 'B']['Value'].sum()
    TSP = data[data['Category'] == 'S']['Value'].sum()

    sdr = TSP/TBP
    sdr_df = pd.concat([sdr_df, pd.DataFrame({'Period': [period], 'SDR': [sdr]})], ignore_index=True)

sdr_df

C:\Users\sdash\AppData\Local\Temp\ipykernel_420\596540504.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sdr_df = pd.concat([sdr_df, pd.DataFrame({'Period': [period], 'SDR': [sdr]})], ignore_index=True)


,Period,SDR
0,1,2.094253


## Floyd-Warshall (Distance Based)

In [186]:
distdf

,ID,1,2,3,4,5,6,7,8,9,10
0,1,0,2,0,0,0,0,0,0,0,0
1,2,2,0,8,12,0,16,0,0,0,0
2,3,0,8,0,0,0,0,0,0,0,22
3,4,0,12,0,0,10,0,0,0,0,0
4,5,0,0,0,10,0,0,0,0,0,4
5,6,0,16,0,0,0,0,14,0,0,0
6,7,0,0,0,0,0,14,0,18,20,0
7,8,0,0,0,0,0,0,18,0,0,6
8,9,0,0,0,0,0,0,20,0,0,0
9,10,0,0,22,0,4,0,0,6,0,0


In [187]:
transaction_df = pd.DataFrame(columns=['Period', 'Seller', 'Buyer', 'Units', 'Unit Price','Savings','Loss'])

In [188]:
def floyd_warshall(graph):
    num_nodes = len(graph)
    # Replace all 0 values with infinity
    graph = np.where(graph == 0, np.inf, graph)
    distance_matrix = np.array(graph, dtype=float)
    intermediate_nodes = np.zeros((num_nodes, num_nodes), dtype=int)

    # Initialize intermediate nodes matrix
    for i in range(num_nodes):
        for j in range(num_nodes):
            intermediate_nodes[i, j] = j if i != j and graph[i, j] != np.inf else -1

    # Floyd-Warshall algorithm with intermediate nodes tracking
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if distance_matrix[i, j] > distance_matrix[i, k] + distance_matrix[k, j]:
                    distance_matrix[i, j] = distance_matrix[i, k] + distance_matrix[k, j]
                    intermediate_nodes[i, j] = intermediate_nodes[i, k]

    np.fill_diagonal(distance_matrix, np.inf)

    return distance_matrix, intermediate_nodes

distdf.set_index('ID', inplace=True)


distance_matrix, intermediate_nodes = floyd_warshall(distdf.values)
short_dist_df = pd.DataFrame(distance_matrix, columns=distdf.columns, index=distdf.index)

print("Shortest Distance Matrix:")
short_dist_df

Shortest Distance Matrix:


,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,
1,inf,2.0,10.0,14.0,24.0,18.0,32.0,34.0,52.0,28.0
2,2.0,inf,8.0,12.0,22.0,16.0,30.0,32.0,50.0,26.0
3,10.0,8.0,inf,20.0,26.0,24.0,38.0,28.0,58.0,22.0
4,14.0,12.0,20.0,inf,10.0,28.0,38.0,20.0,58.0,14.0
5,24.0,22.0,26.0,10.0,inf,38.0,28.0,10.0,48.0,4.0
6,18.0,16.0,24.0,28.0,38.0,inf,14.0,32.0,34.0,38.0
7,32.0,30.0,38.0,38.0,28.0,14.0,inf,18.0,20.0,24.0
8,34.0,32.0,28.0,20.0,10.0,32.0,18.0,inf,38.0,6.0
9,52.0,50.0,58.0,58.0,48.0,34.0,20.0,38.0,inf,44.0


In [189]:
start_node = 1
end_node = 9
route = [str(start_node)] 
current_node = start_node

while current_node != end_node:
    next_node = intermediate_nodes[current_node-1, end_node-1] 
    if next_node == -1:
        print(f"No route from {start_node} to {end_node}")
        break
    route.append(str(next_node + 1)) 
    current_node = next_node + 1 

print(f"Shortest Route from {start_node} to {end_node}: {' -> '.join(route)}")

Shortest Route from 1 to 9: 1 -> 2 -> 6 -> 7 -> 9


# Algorithm

In [190]:
iter_sharpe = []

In [191]:
data = []
columns = ['Period', 'V', 'Del', 'Lij', 'Spij', 'Pij', 'Qij', 'Pg', 'Qg', 'Pi','Qi','Iij','Pl']

In [192]:
def calculate_sdr_price(sdr, buyer_UBP):
    maxi_price = buyer_UBP
    mid_value = (maxi_price + mini_price) / 2
    b = 1 / maxi_price
    a = 1 / mid_value - b
    return round(1 / (a * sdr + b), 2)

def calculate_trading_price(sdr_price, rating_buyer, rating_seller, buyer_rating_pref, buyer_profit_pref, seller_rating_pref, seller_profit_pref, trading_units, buyer_UBP,p):
    price_buyer = sdr_price*(1 +  p[0] * (rating_seller * buyer_rating_pref -  buyer_profit_pref)) 
    price_seller = sdr_price*(1 - p[0] * (rating_buyer * seller_rating_pref - seller_profit_pref))
    trading_price = round((price_buyer + price_seller) / 2, 2)
    trading_price = (mini_price + 0.01) if trading_price < mini_price else trading_price
    trading_price = (buyer_UBP - 0.01) if trading_price > buyer_UBP else trading_price
    return trading_price

def calculate_utilities(trading_unit, buyer_UBP, trading_price, rating_buyer, rating_seller, buyer_rating_pref,
                        buyer_profit_pref, seller_rating_pref, seller_profit_pref, seller_dsm, buyer,seller, dist_factor, bpm,spm):
    buyer_profit = round(trading_unit * (buyer_UBP - trading_price),3)
    seller_profit = round(trading_unit * (trading_price - mini_price),3)
    
    m = (bpm+spm)/2
    
    buyer_util_profit = round(buyer_profit_pref * (buyer_UBP - trading_price), 3)
    buyer_util_rating = round(m * buyer_rating_pref * rating_seller, 3)
    
    seller_util_profit = round(seller_profit_pref * (trading_price - mini_price), 3)
    seller_util_rating = round(m * rating_buyer * seller_rating_pref , 3)
    
    dist_factor = round(dist_factor.loc[buyer,seller],3)
    buyer_utility = round(buyer_util_profit + buyer_util_rating -  dist_factor,3)
    seller_utility = round(seller_util_profit + seller_util_rating -  dist_factor,3)
    
    return buyer_utility, seller_utility, buyer_profit, seller_profit

def create_transaction_df(period, seller, buyer, sdr_price, trading_price, trading_units, seller_profit,
                          buyer_profit, seller_utility, buyer_utility, max_utility):
    return pd.DataFrame({
        'Period': [period],
        'Seller': [seller],
        'Buyer': [buyer],
        'SDR Price': [sdr_price],
        'Trading Price': [trading_price],
        'Trading Energy': [trading_units],
        'Seller Profit': [seller_profit],
        'Buyer Profit': [buyer_profit],
        'Seller Utility': [seller_utility],
        'Buyer Utility': [buyer_utility],
        'Total_Utility': [max_utility]
    })
import pandas as pd

def DistanceFactor(period_data):
    buyers = period_data[period_data['Category'] == 'B']
    sellers = period_data[period_data['Category'] == 'S']

    distance_factors = pd.DataFrame(index=buyers['Prosumer_Id'], columns=sellers['Prosumer_Id'])

    total_shortest_distance = {}
    for seller in sellers['Prosumer_Id']:
        total_shortest_distance[seller] = 0
        for buyer in buyers['Prosumer_Id']:
            total_shortest_distance[seller] += short_dist_df.loc[buyer, seller]

    for buyer in buyers['Prosumer_Id']:
        for seller in sellers['Prosumer_Id']:
            shortest_cumulative_distance = short_dist_df.loc[buyer, seller]
            distance_factors.loc[buyer, seller] = shortest_cumulative_distance / total_shortest_distance[seller]

    return distance_factors


In [193]:
def calculate_utility_matrix_for_period(df, ratings_df, rejected, period,p,bpm, spm):
    total_utilities_df = pd.DataFrame(columns=['Period', 'Seller', 'Buyer', 'Buyer Utility',
                                               'Seller Utility', 'Trading price', 'Trading unit', 'UBP',
                                               'Total_Utility', 'Energy Loss'])
    
    period_data = df[df['Period (Hr)'] == period]
    sdr = sdr_df.loc[sdr_df['Period'] == period, 'SDR'].values[0]
    buyer_data = period_data[period_data['Category'] == 'B']
    seller_data = period_data[period_data['Category'] == 'S']
    distance_factor = DistanceFactor(period_data)
    
    buyer_profit_list = []
    seller_profit_list = []

    for _, row_buyer in buyer_data.iterrows():
        buyer = row_buyer['Prosumer_Id']
        buyer_UBP = row_buyer['Unit Price']
        sdr_price = calculate_sdr_price(sdr, buyer_UBP)
        
        for _, row_seller in seller_data.iterrows():
            seller = row_seller['Prosumer_Id']
            rating_buyer = ratings_df.loc[ratings_df['ID'] == buyer, 'Ratings'].values[0]
            rating_seller = ratings_df.loc[ratings_df['ID'] == seller, 'Ratings_avg'].values[0]
            buyer_profit_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Profit_pref'].values[0]
            buyer_rating_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Rating_pref'].values[0]
            seller_rating_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Rating_pref'].values[0]
            seller_profit_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Profit_pref'].values[0]
            seller_dsm = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'DSM'].values[0]

            trading_unit = round(min(row_buyer['Value'], row_seller['Value']), 3)
            trading_price = calculate_trading_price(sdr_price, rating_buyer, rating_seller, buyer_rating_pref,
                                                    buyer_profit_pref, seller_rating_pref, seller_profit_pref, trading_unit, buyer_UBP,p)
            
            
            buyer_utility, seller_utility, buyer_profit, seller_profit = calculate_utilities(trading_unit, buyer_UBP, trading_price, rating_buyer, rating_seller, buyer_rating_pref,
                        buyer_profit_pref, seller_rating_pref, seller_profit_pref, seller_dsm, buyer, seller, distance_factor,bpm, spm)
            
            if (trading_unit == 0) or ((buyer,seller) in rejected):
                buyer_utility = 0
                seller_utility = 0
                
            buyer_profit_list.append(buyer_profit/trading_unit*buyer_profit_pref)
            seller_profit_list.append(seller_profit/trading_unit*seller_profit_pref)
            
            total_utility = round(buyer_utility * seller_utility, 6)
            

            new_row = create_transaction_df(period, seller, buyer, sdr_price, trading_price, trading_unit,
                                            seller_profit, buyer_profit, seller_utility, buyer_utility, total_utility)
                
            total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)

    utility_matrix = total_utilities_df.pivot_table(index='Seller', columns='Buyer', values='Total_Utility', fill_value=0)
    return utility_matrix, total_utilities_df, mean(buyer_profit_list), mean(seller_profit_list)

def make_transactions_with_priority(df, ratings_df, price_df):
    total_utilities_list = []

    for period in df['Period (Hr)'].unique():
        
        p= [0.4]
        
        limits = np.array([
       [0, 11000, 4000, 3000, 2000, 1000, 1000, 800, 800, 800], #0
        [5000, 0, 4000, 4000, 2000, 3500, 1000, 1000, 800, 500], #1
        [4000, 22, 0, 3000, 2000, 1000, 1000, 900, 800, 1800], #2
        [31, 32, 33, 0, 2500, 1000, 1000, 1000, 800, 800], #3
        [41, 42, 0, 44, 0, 1000, 1000, 1000, 800, 2500], #4
        [51, 52, 0, 54, 55, 0, 2000, 1000, 800, 800], #5
        [61, 62, 63, 0, 65, 66,0, 1500, 2500, 1000], #6
        [71, 72, 73, 74, 75, 76, 77, 0, 800, 2500], #7
        [81, 82, 83, 84, 85, 86, 87, 88, 0,  1000], #8
        [91, 92, 93, 94, 95, 96, 97, 98, 99, 0]
        ])
        rejected = []
        busdata = create_bus_data(df[df['Period (Hr)']==period])
        
        sdr = sdr_df.loc[sdr_df['Period'] == period, 'SDR'].values[0]
        utility_matrix, total_utilities_df, bpm, spm = calculate_utility_matrix_for_period(df, ratings_df, rejected, period,p,3,3)
        utility_matrix, total_utilities_df, _b, _s = calculate_utility_matrix_for_period(df, ratings_df, rejected, period,p,bpm,spm)
        period_data = df[df['Period (Hr)'] == period].copy()
        buyer_data = period_data[period_data['Category'] == 'B']
        
        seller_data = period_data[period_data['Category'] == 'S']
        distance_factor = DistanceFactor(period_data)
        
        #TRANSACTION
        
        while True:
            max_utility = utility_matrix.values.max()
            if max_utility <= 0:
                break
                
            seller, buyer = np.unravel_index(np.argmax(utility_matrix.values), utility_matrix.shape)
            seller, buyer = utility_matrix.index[seller], utility_matrix.columns[buyer]

            buyer_UBP = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Unit Price'].values[0]
            sdr_price = calculate_sdr_price(sdr, buyer_UBP)
                
                
            rating_buyer = ratings_df.loc[ratings_df['ID'] == buyer, 'Ratings'].values[0]
            rating_seller = ratings_df.loc[ratings_df['ID'] == seller, 'Ratings_avg'].values[0]
            buyer_profit_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Profit_pref'].values[0]
            buyer_rating_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Rating_pref'].values[0]
            seller_rating_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Rating_pref'].values[0]
            seller_profit_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Profit_pref'].values[0]
            seller_dsm = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'DSM'].values[0]
            
            row_buyer = period_data[period_data['Prosumer_Id'] == buyer]
            row_seller = period_data[period_data['Prosumer_Id'] == seller]
            
            trading_units = round(min(row_buyer['Value'].values[0], row_seller['Value'].values[0]), 3)
            trading_price = calculate_trading_price(sdr_price, rating_buyer, rating_seller, buyer_rating_pref,
                                                    buyer_profit_pref, seller_rating_pref, seller_profit_pref, trading_units, buyer_UBP,p)
        
            buyer_utility, seller_utility, buyer_profit, seller_profit = calculate_utilities(trading_units, buyer_UBP, trading_price, rating_buyer, rating_seller, buyer_rating_pref,
                        buyer_profit_pref, seller_rating_pref, seller_profit_pref, seller_dsm, buyer, seller, distance_factor,bpm,spm)
            
            if (trading_units == 0) or ((buyer,seller) in rejected):
                buyer_utility = 0
                seller_utility = 0

            total_utility = round(buyer_utility * seller_utility, 6)
            
            if (buyer,seller) in rejected:
                continue
                
            if trading_units <= 0:
                utility_matrix[buyer] = 0
                utility_matrix.loc[seller] = 0
                continue
                
            
            period_data.loc[period_data['Prosumer_Id'] == seller, 'Value'] -= trading_units
            period_data.loc[period_data['Prosumer_Id'] == buyer, 'Value'] -= trading_units
            
            
            #POWER FLOW
            busdata.loc[busdata['Bus']==seller, 'Type'] = 2
            busdata.loc[busdata['Bus']==seller, 'PGi'] += (trading_units*1000*4)
            busdata_array = busdata.to_numpy()
            power_flow = PowerFlow(busdata_array, linedata) 
            V, Del, Iter, check = power_flow.loadflow(busdata_array[:, 2], busdata_array[:, 3], BMva)
            Lij,Sij,Pij,Qij,Pg,Qg, Pi, Qi, Iij, Pl = power_flow.powerflow(V, np.deg2rad(Del), BMva)
            flag = False
            
            #LIMITS CHECKING
            for i in range(max_id):
                for j in range(max_id):
                    if( (i<j) and (limits[i][j] - max(np.abs(Pij[i][j]),np.abs(Pij[j][i])) < 0) or (check == False)):
                        rejected.append((buyer,seller))
                        utility_matrix.loc[seller, buyer] = 0
                        period_data.loc[period_data['Prosumer_Id'] == seller, 'Value'] += trading_units
                        period_data.loc[period_data['Prosumer_Id'] == buyer, 'Value'] += trading_units
                        
                        busdata.loc[busdata['Bus']==seller, 'PGi'] -= (trading_units*1000*4)
                        if busdata.loc[busdata['Bus'] == seller, 'PGi'].values[0] == 0:
                            busdata.loc[busdata['Bus'] == seller, 'Type'] = 3
                            
                        flag = True
                        
                        print(f"Period: {period}, Rejecting Transaction: {buyer}, {seller}, Line Limit: {limits[i][j]}, Line: {i} -> {j}, Current Power Flow ({i}->{j}): {np.abs(Pij[i][j])},  Total Utility {max_utility},  Trading Units {trading_units} ")
                        break
                        
                if flag:
                    break
                        
            if flag:
                continue
                        
            print(f"Period: {period}, Transaction Approved: {buyer}, {seller}")

            
            data.append([period, V, Del, Lij, Sij, Pij, Qij, Pg, Qg, Pi, Qi, Iij, Pl])
            
            if (period_data.loc[period_data['Prosumer_Id'] == seller, 'Value'].values[0] == 0):
                utility_matrix.loc[seller] = 0
            else:
                utility_matrix[buyer] = 0
            
            total_utilities_list.append(create_transaction_df(period, seller, buyer, sdr_price, trading_price,
                                                              trading_units, seller_profit, buyer_profit, seller_utility,
                                                              buyer_utility, max_utility))

            print(f"Transaction: Period {period}, Buyer: {buyer}, Seller: {seller}, Total Utility {max_utility}, Trading Units {trading_units}, Price {trading_price}")

            
            
        # SURPLUS TO GRID
        seller_data = period_data[period_data['Category'] == 'S']
        for _, row_seller in seller_data.iterrows():
            
            if round(row_seller['Value'],3)>0:
                
                seller = row_seller['Prosumer_Id']
                trading_units = round(row_seller['Value'],3)
                
                
                #POWER FLOW
                busdata.loc[busdata['Bus']==seller, 'Type'] = 2
                busdata.loc[busdata['Bus']==seller, 'PGi'] += (trading_units*1000*4)
                busdata_array = busdata.to_numpy()
                power_flow = PowerFlow(busdata_array, linedata) 
                V, Del, Iter, check = power_flow.loadflow(busdata_array[:, 2], busdata_array[:, 3], BMva)
                Lij,Sij,Pij,Qij,Pg,Qg, Pi, Qi, Iij, Pl = power_flow.powerflow(V, np.deg2rad(Del), BMva)
                
                
                for i in range(max_id):
                    for j in range(max_id):
                        if((i<j) and (limits[i][j] - max(np.abs(Pij[i][j]),np.abs(Pij[j][i])) < 0) or (check == False)):
                            rejected.append((buyer,seller))
                            utility_matrix.loc[seller, buyer] = 0
                            busdata.loc[busdata['Bus']==seller, 'PGi'] -= (trading_units*1000*4)
                            if busdata.loc[busdata['Bus'] == seller, 'PGi'].values[0] == 0:
                                busdata.loc[busdata['Bus'] == seller, 'Type'] = 3

                            flag = True
                            print(f"Period: {period}, Rejecting Transaction: 1, {seller}, Line Limit: {limits[i][j]}, Line: {i} -> {j}, Current Power Flow ({i}->{j}): {np.abs(Pij[i][j])}  Total Utility {max_utility},  Trading Units {trading_units}")
                            break
                        
                    if flag:
                        break
                     
                if flag:
                    continue
                      
                
                data.append([period, V, Del, Lij, Sij, Pij, Qij, Pg, Qg, Pi, Qi, Iij, Pl])
                total_utilities_list.append(create_transaction_df(period, seller, 1, mini_price, mini_price,
                                                              trading_units, 0, 0, 0,
                                                              0, 0))
                print(f"Transaction: Period {period},Buyer: 1, Seller: {seller}, Total Utility {max_utility}, Trading Units {trading_units}, Price {mini_price}")
                
                
                
                
        #GRID TO DEMAND
        buyer_data = period_data[period_data['Category'] == 'B']
        for _, row_buyer in buyer_data.iterrows():
            
            if round(row_buyer['Value'],3)>0:
                buyer = row_buyer['Prosumer_Id']
                trading_units = round(row_buyer['Value'],3)
                UBP = row_buyer['Unit Price']
                total_utilities_list.append(create_transaction_df(period, 1, buyer, UBP, UBP,
                                                              trading_units, 0, 0, 0,
                                                              0, 0))
                print(f"Transaction: Period {period},Buyer: {buyer}, Seller: 1, Total Utility {max_utility}, Trading Units {trading_units}, Price {UBP}")
            
    total_utilities_df = pd.concat(total_utilities_list, ignore_index=True)
    return total_utilities_df


start_time = time.time()
result_transactions = make_transactions_with_priority(df, ratings_df, sdr_df)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

# GRID power flow increase - 1000 watts, 1200 - limit
# trading power flow decrease




C:\Users\sdash\AppData\Local\Temp\ipykernel_420\929919774.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)
C:\Users\sdash\AppData\Local\Temp\ipykernel_420\929919774.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)


Period: 1, Rejecting Transaction: 4, 5, Line Limit: 2500, Line: 3 -> 4, Current Power Flow (3->4): 2823.4154204777883,  Total Utility 1.088503,  Trading Units 0.706 
Period: 1, Transaction Approved: 4, 2
Transaction: Period 1, Buyer: 4, Seller: 2, Total Utility 0.149625, Trading Units 0.334, Price 5.08
Period: 1, Transaction Approved: 4, 3
Transaction: Period 1, Buyer: 4, Seller: 3, Total Utility 0.009088, Trading Units 0.536, Price 4.75
Transaction: Period 1,Buyer: 1, Seller: 3, Total Utility 0, Trading Units 0.246, Price 3.6
Period: 1, Rejecting Transaction: 1, 5, Line Limit: 2500, Line: 3 -> 4, Current Power Flow (3->4): 2823.396460454284  Total Utility 0,  Trading Units 0.706
Elapsed time: 0.2936434745788574 seconds


In [194]:
result_transactions.to_csv('result_transactions.csv', index=False)
result_transactions


,Period,Seller,Buyer,SDR Price,Trading Price,Trading Energy,Seller Profit,Buyer Profit,Seller Utility,Buyer Utility,Total_Utility
0,1,2,4,5.04,5.08,0.334,0.494,1.643,0.125,1.197,0.149625
1,1,3,4,5.04,4.75,0.536,0.616,2.814,0.008,1.136,0.009088
2,1,3,1,3.60,3.60,0.246,0.000,0.000,0.000,0.000,0.000000


In [195]:
result_transactions['Trading Energy'].sum()

1.116